In [1]:
# Installed
import numpy as np
import theano
import theano.tensor as TT
from pdb import set_trace
theano.config.optimizer='None'
theano.gof.cc.get_module_cache().clear()

# Local
import shared
import misc 
reload(misc)
import nef
reload(nef)
import gru

In [2]:
# outer world
nr_words = 40     # Number of words
voc_size = 100    # Vocabulary size 
emb_size = 50     # Embedding size
nr_classes = 2    # Number of classes
# 
# Sketch matrix
sketch_dim = 10   # Sketch dimension
nr_sketch = 3     # Sketch number
# Attention
attention_dim = 6 # Attention dimension
conv_dim = 3      # Attention convolutionsl dimension 

# Train param
lrate = 1.0

In [3]:
H = TT.matrix()
y = nef.easy_first(H, nr_sketch, attention_dim, sketch_dim, conv_dim, nr_classes)

In [4]:
param = misc.get_param(y)
param

[inference.w_p,
 attention.W_sz,
 attention.W_hz,
 attention.v,
 sketch.W_s,
 sketch.W_ss,
 sketch.W_hs,
 inference.W_sp]

In [5]:
# DEBUG
data_H = np.random.randn(sketch_dim, nr_words)
data_S = np.random.randn(sketch_dim, nr_words)

In [6]:
dbg = theano.function([H], y)
print [dat.shape for dat in dbg(data_H)]

[(40,), (40,)]


In [7]:
# Complete system

In [8]:
# Embeddings
E = shared.random(size=(emb_size, voc_size), name='E')
# GRU on the source
param_GRU = gru.init_gru(emb_size, sketch_dim, bias=True)

In [9]:
# Input
x = TT.ivector('x')
# Ground truth
y = TT.ivector('x')
# Embedding
emb = E[:, x]
# RNN
H = gru.gru(emb.T, param_GRU).T
# Easy first 
p_y = nef.easy_first(H, nr_sketch, attention_dim, sketch_dim, conv_dim, nr_classes)

In [17]:
param = misc.get_param(p_y)
param

[inference.w_p,
 b_U,
 U,
 b_Ur,
 U_r,
 b_Uz,
 U_z,
 <TensorType(float64, matrix)>,
 [b b_r b_z],
 [W, W_r, W_z],
 E,
 attention.W_sz,
 attention.W_hz,
 attention.v,
 sketch.W_s,
 sketch.W_ss,
 sketch.W_hs,
 inference.W_sp]

In [11]:
# Fake data
data_x = np.random.randint(0, voc_size-1, size=(nr_words)).astype('int32')
data_y = np.random.randint(0, nr_classes-1, size=(nr_words)).astype('int32')

In [12]:
dbg = theano.function([x], p_y)
print [dat.shape for dat in dbg(data_x)]

[(40,), (40,)]


In [13]:
cost = -TT.mean(TT.log(p_y)[y, TT.arange(y.shape[0])])

In [14]:
dbg = theano.function([x, y], cost)

In [16]:
print dbg(data_x, data_y)

0.693147187634


In [18]:
updates = [(par, par - lrate*TT.grad(cost, par)) for par in param]

In [19]:
batch_train = theano.function([x, y], cost, updates=updates)

In [25]:
batch_train(data_x, data_y)

array(0.0900804456359474)